In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
import os
import pandas as pd
import numpy as np
import random
import torch.nn.init as init
import pdb

In [4]:
seed = 100

def file_name(file_dir,file_type='.csv'):#默认为文件夹下的所有文件
    lst = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if(file_type == ''):
                lst.append(file)
            else:
                if os.path.splitext(file)[1] == str(file_type):#获取指定类型的文件名
                    lst.append(file)
    return lst

def normalize0(inputs):
    normalized = []
    for eq in inputs:
        maks = np.max(np.abs(eq))
        if maks != 0:
            normalized.append(eq / maks)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize1(inputs):
    normalized = []
    for eq in inputs:
        mean = np.mean(eq)
        std = np.std(eq)
        if std != 0:
            normalized.append((eq - mean) / std)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize(inputs):
    normalized = []
    for eq in inputs:
        with np.errstate(invalid='ignore'):
            eq_log = [np.log(x) if i < 5 else x for i, x in enumerate(eq)]
            eq_log1 = np.nan_to_num(eq_log).tolist()
            normalized.append(eq_log1)
    return np.array(normalized)


def k_fold_split(inputs, targets, K, seed=None):
    # 确保所有随机操作都使用相同的种子
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

    ind = int(len(inputs) / K)
    inputsK = []
    targetsK = []

    for i in range(0, K - 1):
        inputsK.append(inputs[i * ind:(i + 1) * ind])
        targetsK.append(targets[i * ind:(i + 1) * ind])

    inputsK.append(inputs[(i + 1) * ind:])
    targetsK.append(targets[(i + 1) * ind:])

    return inputsK, targetsK


def merge_splits(inputs, targets, k, K):
    if k != 0:
        z = 0
        inputsTrain = inputs[z]
        targetsTrain = targets[z]
    else:
        z = 1
        inputsTrain = inputs[z]
        targetsTrain = targets[z]

    for i in range(z + 1, K):
        if i != k:
            inputsTrain = np.concatenate((inputsTrain, inputs[i]))
            targetsTrain = np.concatenate((targetsTrain, targets[i]))

    return inputsTrain, targetsTrain, inputs[k], targets[k]


def targets_to_list(targets):
    targetList = np.array(targets)

    return targetList


In [19]:
class GCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        reg_const=0.0001
        self.gcn1 = GCNConv(dim_in, dim_h,weight_decay=reg_const,add_self_loops=True)
        self.gcn2 = GCNConv(dim_h, dim_h,weight_decay=reg_const,add_self_loops=True)
        self.gcn3 = GCNConv(dim_h, dim_h,bias=True,weight_decay=reg_const,add_self_loops=True)
        self.linear2 = torch.nn.Linear(dim_h, dim_out).double()        
        self.initialize_weights()

    def initialize_weights(self):
        for layer in [self.gcn1, self.gcn2, self.gcn3, self.linear2]:
            if isinstance(layer, torch.nn.Linear):
                init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)
            elif isinstance(layer, GCNConv):
                # For GCNConv, handle weight and bias separately
                init.xavier_uniform_(layer.lin.weight)
                if layer.lin.bias is not None:
                    init.zeros_(layer.lin.bias)




#     def forward(self, x, adj):
#         x=x.double()
#         edge_index = torch.stack([np.nonzero(t)for t in torch.unbind(adj,dim=0)],dim=0).permute(0,2,1)
#         self.gcn1 = self.gcn1.to(torch.double)
#         self.gcn2 = self.gcn2.to(torch.double)
#         self.gcn3 = self.gcn3.to(torch.double)
#         results = []
#         for i in range(x.size(0)):
#             h = self.gcn1(x[i], edge_index[i])
#             h = torch.nn.LeakyReLU()(h)
#             h = self.gcn2(h, edge_index[i])
#             h = torch.nn.LeakyReLU()(h)
#             h = self.gcn3(h, edge_index[i])
#             h = torch.nn.LeakyReLU()(h)
#             last_column = h[:, -1]
#             conv1_new = last_column.unsqueeze(-1)
#             #conv1_new = torch.nn.Flatten(conv1_new)
#             #conv1_new = F.dropout(conv1_new,p=0.3)
#             conv1_new = F.dropout(conv1_new.view(conv1_new.size(0), -1), p=0.3, training=self.training)        
#             y_hat = self.linear2(conv1_new.T)
#             results.append(y_hat)
#         return torch.stack(results, dim=0)

    def forward(self, x, adj):
        x = x.double()
        edge_indices = []
        for i in range(x.size(0)):
            # 获取当前样本的 edge_index
            edge_index = torch.nonzero(adj[i]).t().contiguous()
            edge_indices.append(edge_index)

        # 将 edge_index 列表堆叠为一个张量
        edge_indices = torch.stack(edge_indices, dim=0)
        print(edge_indices.shape)

        # 在整个 batch 上应用 GCN
        h = self.gcn1(x, edge_indices)
        h = torch.nn.LeakyReLU()(h)
        h = self.gcn2(h, edge_indices)
        h = torch.nn.LeakyReLU()(h)
        h = self.gcn3(h, edge_indices)
        h = torch.nn.LeakyReLU()(h)

        # 在最后一个维度上应用 dropout
        h = F.dropout(h, p=0.3, training=self.training)

        # 展平最后两个维度
        conv1_new = h.view(h.size(0), -1)

        # 应用线性层
        y_hat = self.linear2(conv1_new.T)


        #conv1_new = F.dropout(conv1_new.view(conv1_new.size(0), -1), p=0.3, training=self.training)

        y_hat = self.linear2(conv1_new.T)
        return y_hat



    def fit(self,train_loader, val_loader,num_epochs=5000, patience=50):
        #best_val_loss = float('inf')
        best_val_loss = torch.tensor(float('inf'), dtype=torch.double)
        patience_counter = 0
        optimizer = optim.RMSprop(self.parameters(), lr=0.00015,weight_decay=0.)
        #criterion = nn.MSELoss()
        # 使用平滑的 L1 损失，也称为 Huber loss
        criterion = nn.SmoothL1Loss()

        torch.autograd.set_detect_anomaly(True)
        for epoch in range(num_epochs):
            self.train()
            for inputs, graph_input, targets in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs, graph_input)
                loss = criterion(outputs, targets)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm = 1)
                optimizer.step()

            self.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_graph_input, val_targets in val_loader:
                    val_outputs = self(val_inputs, val_graph_input)
                    val_loss += criterion(val_outputs, val_targets).item()

            val_loss /= len(val_loader)
            if (epoch + 1) % 20 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}')

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    return
                
                
    def test(self,test_loader):
        self.eval()
        predictions = []
        for test_inputs, test_graph_input, _ in test_loader:
            batch_predictions = self(test_inputs, test_graph_input)
            predictions.append(batch_predictions)
        predictions = torch.cat(predictions)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming your PyTorch model is defined as before


if __name__ == "__main__":
    # stock_info = sys.argv[0]
    # lag_bin = int(sys.argv[1])
    # lag_day = int(sys.argv[2])
    # bin_num = int(sys.argv[3])
    # random_state_here = int(sys.argv[4])
    # test_set_size = float(sys.argv[5])
    lag_bin = 3
    lag_day = 3
    num_nodes = (int(lag_bin)+1)*(int(lag_day)+1)
    forecast_days = 15
    bin_num=24
    random_state_here = 88
    mape_list = []
    data_dir = './data/volume/0308/'
    files =file_name('./data/')
    stocks_info = list(set(s.split('_25')[0] for s in files))
    print(stocks_info)
    for stock_info in stocks_info[0:2]:
        print(f'>>>>>>>>>>>>>>>>>>>>{stock_info}>>>>>>>>>>>>>>>>>>>>>>>')
        data_dir1 = f'{data_dir}{stock_info}_{lag_bin}_{lag_day}'
        test_set_size = bin_num*forecast_days
        K = 5
        inputs_data = np.load(f'{data_dir1}_inputs.npy', allow_pickle=True)#.astype(np.float64)
        #inputs_data = [[[float(x) for x in sublist] for sublist in list1] for list1 in inputs_data]
        inputs_data = [[[torch.tensor(x, dtype=torch.float64) for x in sublist] for sublist in list1] for list1 in inputs_data]
        array_data = np.array(inputs_data)
        inputs = np.reshape(array_data, (len(inputs_data), num_nodes,-1))
        targets = np.load(f'{data_dir1}_output.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.load(f'{data_dir1}_graph_input.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.array([graph_input] * inputs.shape[0])
        #graph_input = np.nonzero(graph_input)
        #graph_input = torch.tensor(graph_input,dtype=torch.float64)
        graph_features = np.load(f'{data_dir1}_graph_coords.npy', allow_pickle=True).astype(np.float64)
        graph_features = np.array([graph_features] * inputs.shape[0])

        trainInputs, testInputs, traingraphInput, testgraphInput, traingraphFeature, testgraphFeature, trainTargets, testTargets = train_test_split(inputs, graph_input, graph_features, targets, test_size=test_set_size, 
                                                     random_state=random_state_here)
        testInputs = normalize(testInputs)
        # testInputs = test_inputs
        inputsK, targetsK = k_fold_split(trainInputs, trainTargets, K)

        mape_list = []

        test_dataset = TensorDataset(torch.tensor(testInputs), torch.tensor(testgraphInput), torch.tensor(testTargets))
        test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
        K = 5  # Number of folds
        for k in range(K):
            torch.manual_seed(0)  # Set a random seed for reproducibility

            trainInputsAll, trainTargets, valInputsAll, valTargets = merge_splits(inputsK, targetsK, k, K)

            trainGraphInput = traingraphInput[0:trainInputsAll.shape[0], :]
            trainGraphFeatureInput = traingraphFeature[0:trainInputsAll.shape[0], :]

            valGraphInput = traingraphInput[0:valInputsAll.shape[0], :]
            valGraphFeatureInput = traingraphFeature[0:valInputsAll.shape[0], :]

            trainInputs = normalize(trainInputsAll[:, :])
            valInputs = normalize(valInputsAll[:, :])

            # Assuming trainInputs, trainGraphInput, trainGraphFeatureInput, trainTargets are PyTorch tensors
            train_dataset = TensorDataset(torch.tensor(trainInputs), torch.tensor(trainGraphInput),torch.tensor(trainTargets))
            val_dataset = TensorDataset(torch.tensor(valInputs), torch.tensor(valGraphInput),torch.tensor(valTargets))

            # train_dataset = TensorDataset(torch.tensor(trainInputs, dtype=torch.float32), torch.tensor(trainGraphInput, dtype=torch.float32), torch.tensor(trainGraphFeatureInput, dtype=torch.float32), torch.tensor(trainTargets, dtype=torch.float32))
            # val_dataset = TensorDataset(torch.tensor(valInputs, dtype=torch.float32), torch.tensor(valGraphInput, dtype=torch.float32), torch.tensor(valGraphFeatureInput, dtype=torch.float32), torch.tensor(valTargets, dtype=torch.float32))

            train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)

            # Instantiate the PyTorch model
        
            model = GCN(7,valInputs.shape[1],1)
            print()
            model.fit(train_loader, val_loader)
            predictions=model.test(test_loader)
            # Define optimizer and loss function
            #optimizer = optim.RMSprop(model.parameters(), lr=0.00015)
           # criterion = nn.MSELoss()

            # Train the model
            # train_model(model, train_loader, val_loader, criterion, optimizer)

            # Save the PyTorch model
            torch.save(model.state_dict(), f'models/{stock_info}_{lag_bin}_{lag_day}_gcn_model_iteration_{k}.pt')
            print(model.state_dict())
            # Evaluate on the test set
            # model.eval()
            # with torch.no_grad():
            #     predictions = []
            #     for test_inputs, test_graph_input, test_graph_feature, _ in test_loader:
            #         batch_predictions = model(test_inputs, test_graph_input, test_graph_feature)
            #         predictions.append(batch_predictions)
            #     predictions = torch.cat(predictions)

            # Perform any necessary operations with the predictions
            # ...
            print()
            print('Fold number:', k)
            print('Total number of epochs ran =', len(history))  # This is a PyTorch equivalent

            new_predictions = np.array(predictions)
            print(type(new_predictions))
            print(new_predictions.shape)
            # new_predictions = new_predictions.squeeze().tolist()
            new_predictions = [item for sublist in new_predictions for item in sublist]
            MAPE = []

            # MAPE.append(mean_absolute_percentage_error(trainTargets[:], new_predictions[:]))
            # testTargets0 = [item for sublist in trainTargets for item in sublist]
            print(testTargets)
            print(new_predictions)
            MAPE.append(mean_absolute_percentage_error(testTargets[:], new_predictions[:]))
            print(MAPE)
            # testTargets0 = [item for sublist in testTargets for item in sublist]
            testTargets0 = list(testTargets)

            res = {
                'testTargets': testTargets0,
                'new_predictions': new_predictions
            }

            res_df = pd.DataFrame(res)
            res_df.to_csv(f'./result/{stock_info}_{lag_bin}_{lag_day}_res_test_MAPE{k}.csv', index=False)

            print('MAPE = ', np.array(MAPE).mean())
            MAPE_mean = np.array(MAPE).mean()
            mape_list.append(MAPE)

        print('-')
        print('mape score = ', mape_list)


['603095_XSHG', '000046_XSHE', '600622_XSHG', '002282_XSHE', '002841_XSHE', '300263_XSHE', '300174_XSHE', '603359_XSHG', '000753_XSHE', '300343_XSHE', '000998_XSHE', '300540_XSHE', '300133_XSHE', '002882_XSHE', '000951_XSHE', '603053_XSHG', '300433_XSHE', '002679_XSHE']
>>>>>>>>>>>>>>>>>>>>603095_XSHG>>>>>>>>>>>>>>>>>>>>>>>


/tmp/ipykernel_2182843/740401852.py:41: RuntimeWarning: divide by zero encountered in log
  eq_log = [np.log(x) if i < 5 else x for i, x in enumerate(eq)]



torch.Size([50, 2, 42])


IndexError: too many indices for tensor of dimension 1

In [ ]:
x=np.random.randint(0,10,size=50*16*16).reshape([50,16,16])

In [61]:
np.shape(torch.stack([np.nonzero(t)for t in torch.unbind( torch.randn(10,20,30),dim=0)],dim=0))

torch.Size([10, 600, 2])